In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data99666


In [1]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [2]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: cannot create directory ‘/home/aistudio/external-libraries’: File exists
Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 122kB 14.8MB/s eta 0:00:01


In [3]:
#!unzip -oq /home/aistudio/data/data99666/A_Pred.zip -d data/
!unzip -oq /home/aistudio/data/data99666/dataset.zip -d data/
#!unzip -oq /home/aistudio/data/data99666/paddle-seg-semi1.zip -d external-libraries

#注意，第一次运行若在external-libraries目录下没有paddle_seg-semi1的文件夹，请取消第三行的注释，然后会在external-libraries下面生成一个paddle-seg-semi1的文件，
#麻烦一下rename 成paddle_seg-semi1，这是paddleseg源码被我修改后的，我添加了自己的网络，损失函数等.

In [4]:
# 解压B榜测试集
!unzip -oq /home/aistudio/data/data99666/B榜测试数据集.zip -d data

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')
sys.path.append('/home/aistudio/external-libraries/paddle_seg_semi1') # 我额外加的

本次比赛，我们队伍一共使用了4个模型，通过四个模型的加权融合后得出最好的结果，下面分别介绍每个模型

这些都是基于paddleseg套件进行修改的，所以按照paddleseg的方法就行模型训练和预测

与其说是四个模型，不如说是3种骨干网络，2种解码网络

四个模型的模型文件可在  

external-libraries/paddle_seg_semi1/paddleseg/models/FMFNetwork.py

external-libraries/paddle_seg_semi1/paddleseg/models/ACFFNetwork.py

external-libraries/paddle_seg_semi1/paddleseg/models/ACFF_ViT.py

external-libraries/paddle_seg_semi1/paddleseg/models/Next_ACFF.py

看到

四个配置文件可在external-libraries/paddle_seg_semi1/configs/semi 看到

分别叫 fcn_FMF_semi_50k.yml、fcn_ACFFNet_semi_50k.yml、 fcn_ACFFViT.yml、 fcn_NextACFF.yml 

数据处理没啥可看的 可在external-libraries/paddle_seg_semi1/paddleseg/datasets/semi.py 看到

**另外，严正声明，比赛所用的模型，除骨干网络外，皆为本人所提出的模型算法，其中ACFFNetwork.py中除骨干网络外的模型为本人所投sci（在大修）文章中的模型稍加修改后的模型， FMFNet中除骨干网络外的模型为本人正在写sci的模型稍加修改后的模型，
与baseline 没一点相同**

**另外，本人十分愿意为paddleseg提供比赛冠军模型，成为PPDE**

因为4个模型，所以预测时间略长

In [2]:
# 下面详解每个模型的训练与预测

In [ ]:
################### FMF ########################

In [ ]:
# 这是我自己添加的第一个模型及其配置文件，模型的具体细节可以在 /home/aistudio/external-libraries/paddle_seg_semi1/paddleseg/models/FMFNetwork.py 这个目录里看到
# 简要介绍一下模型、损失函数、数据增强、学习率和训练策略以及出现的一些问题
# 模型：我们采用FCN架构，以paddlecls中提供的res2net200以及其预训练权重做为骨干网络来提取特征信息，输入数据为3×256×256的图片，四层layer我们分别提取了256×64×64、 512×32×32 、 1024×16×16、 2048×8×8的特征信息
     # 本次比赛的要求是生成具有目标的单通道图，一般的做法为四层特征信息的融合，即encoder-decoder架构，我们也是采用这样的方法，只不过在中间添加了特征优化的模块，可见 FMFNetwork.py中的语义补充模块（SSM），特征交互模块（FIM），
     # 关于这俩模块我将会在另外的方案分享里具体描述，本次提交就不做过细讲解.四层特征经过融合后使用3×3卷积调整为1通道，经过sigmoid函数与上采样恢复到原尺寸输出后在loss函数（一会细讲）和Momentum优化器的优化下进行训练.
# 损失函数：我们自己实现了iou损失函数在2分类中的实现 代码见：external-libraries/paddle_seg_semi1/paddleseg/models/losses/semi_loss.py
# 数据增强：根据paddleseg的用法，我们并没有用过多的数据增强操作，只用了resize、 对比度亮度、水平翻转、垂直反转、随即模糊、归一化这些操作，训练数据使用随机种子固定随机数，随机选取2K个作为验证集.
# 学习率：使用paddleseg推荐的poly decay 作为学习率衰减策略，初始学习率为0.01，最小学习率为0.0001，
# 训练策略以及一些问题：训练策略如下，2000次迭代保存一次模型，本来打算训练2w4k次迭代，但是发现模型在1w2k次迭代效果最好，但是loss和评价指标却一直在变好，可能是存在验证集过拟合的现象，具体原因不知.

# 这是训练代码，复现请配置好文件后直接运行下一个代码块
%cd external-libraries/paddle_seg_semi1/
!python train.py --config configs/semi/fcn_FMF_semi_50k.yml \
--do_eval \
--save_dir /home/aistudio/output/fmf \
--save_interval 2000 \
--log_iters 100  \
--use_vdl

[Errno 2] No such file or directory: 'external-libraries/paddle_seg_semi1/'
/home/aistudio/external-libraries/paddle_seg_semi1
/home/aistudio/external-libraries/paddle_seg_semi1/paddleseg/models/ACFF_ViT.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable
2021-08-03 12:09:25 [INFO]	
------------Environment Information-------------
platform: Linux-4.4.0-150-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Cuda compilation tools, release 10.1, V10.1.243
cudnn: 7.6
GPUs used: 1
CUDA_VISIBLE_DEVICES: None
GPU: ['GPU 0: Tesla V100-SXM2-32GB']
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~16.04) 7.5.0
PaddlePaddle: 2.1.0
OpenCV: 4.1.1
------------------------------------------------
2021-08-03 12:09:27 [INFO]	
---------------Config Information---------------
batch_size: 32
iters: 36000

In [ ]:
# 这是预测的代码，依然是根据paddleseg套件直接运行的，采取了多尺度预测和水平翻转的预测策略， 这里就没什么好讲的了
%cd external-libraries/paddle_seg_semi1/
!python predict.py --config configs/semi/fcn_FMF_semi_50k.yml \
--model_path /home/aistudio/data/data99666/200_FMF.pdparams \
--image_path /home/aistudio/data/test_image/ \
--save_dir /home/aistudio/output/result_fmf \
--aug_pred --scales  1 1.5 2 \
--flip_horizontal 

[Errno 2] No such file or directory: 'external-libraries/paddle_seg_semi1/'
/home/aistudio/external-libraries/paddle_seg_semi1
/home/aistudio/external-libraries/paddle_seg_semi1/paddleseg/models/ACFF_ViT.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable
2021-08-06 10:05:12 [INFO]	
---------------Config Information---------------
batch_size: 32
iters: 36000
learning_rate:
  decay:
    end_lr: 0.0001
    power: 0.9
    type: poly
  value: 0.01
loss:
  coef:
  - 1
  types:
  - type: SemiLoss
model:
  backbone:
    pretrained: /home/aistudio/data/data99666/Res2Net200_vd_26w_4s_ssld_pretrained.pdparams
    type: Res2Net
  num_classes: 2
  type: FMFModel
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 0.0005
train_dataset:
  dataset_root: /home/aistudio/data
  mode: train
  transforms:
  - max_rotation: 30
    max_scale_factor: 1.5
    min_scale_fa

In [ ]:
############# ACFF ################

In [ ]:
# 这是我自己添加的第二个模型及其配置文件，模型的具体细节可以在 /home/aistudio/external-libraries/paddle_seg_semi1/paddleseg/models/ACFFNetwork.py 这个目录里看到
# 简要介绍一下模型、损失函数、数据增强、学习率和训练策略以及出现的一些问题
# 模型：我们采用FCN架构，以paddlecls中提供的res2net200以及其预训练权重做为骨干网络来提取特征信息，输入数据为3×256×256的图片，四层layer我们分别提取了256×64×64、 512×32×32 、 1024×16×16、 2048×8×8的特征信息
     # 本次比赛的要求是生成具有目标的单通道图，一般的做法为四层特征信息的融合，即encoder-decoder架构，我们也是采用这样的方法，只不过在中间添加了特征优化的模块，可见 ACFFNetwork.py中的自优化模块（SR），注意力模块（SE）等，
     # 关于这些模块我将会在另外的方案分享里具体描述，本次提交就不做过细讲解.四层特征经过融合后使用3×3卷积调整为1通道，经过sigmoid函数与上采样恢复到原尺寸输出后在loss函数（一会细讲）和Momentum优化器的优化下进行训练.
# 损失函数：我们自己实现了iou损失函数在2分类中的实现 代码见：external-libraries/paddle_seg_semi1/paddleseg/models/losses/semi_loss.py
# 数据增强：根据paddleseg的用法，我们并没有用过多的数据增强操作，只用了resize、 对比度亮度、水平翻转、垂直反转、随即模糊、归一化这些操作，训练数据使用随机种子固定随机数，随机选取2K个作为验证集.
# 学习率：使用paddleseg推荐的poly decay 作为学习率衰减策略，初始学习率为0.01，最小学习率为0.0001，
# 训练策略以及一些问题：训练策略如下，3000次迭代保存一次模型，本来打算训练2w4k次迭代，但是发现模型在1w2k次迭代效果最好，但是loss和评价指标却一直在变好，可能是存在验证集过拟合的现象，具体原因不知.

# 这是训练代码，复现请配置好文件后直接运行下一个代码块
%cd external-libraries/paddle_seg_semi1/
!python train.py --config configs/semi/fcn_ACFFNet_semi_50k.yml \
--do_eval \
--save_dir /home/aistudio/output/acff \
--save_interval 3000 \
--log_iters 100  \
--use_vdl

/home/aistudio/external-libraries/paddle_seg_semi1
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2021-07-29 21:51:18 [INFO]	
------------Environment Information-------------
platform: Linux-4.4.0-166-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: False
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~16.04) 7.5.0
PaddlePaddle: 2.1.0
OpenCV: 4.1.1
------------------------------------------------
2021-07-29 21:51:35 [INFO]	
---------------Config Information---------------
batch_size: 32
iters: 24000
learning_rate:
  decay:
    end_lr: 0.001
    power: 0.9
    type: poly
  value: 0.01
loss:
  coef:
  - 1
  types:
  - ignore_index: 255
    type: SemiLoss
model:
  backbone:
    layers: 200
    pretrained: /home/aistudio/data/data99666/Res2Net200_vd_2

In [ ]:
# 这是预测的代码，依然是根据paddleseg套件直接运行的，采取了多尺度预测和水平翻转的预测策略， 这里就没什么好讲的了
%cd external-libraries/paddle_seg_semi1/
!python predict.py --config configs/semi/fcn_ACFFNet_semi_50k.yml \
--model_path /home/aistudio/data/data99666/res2net200_score_0.80918.pdparams \
--image_path /home/aistudio/data/test_image/ \
--save_dir /home/aistudio/output/result_acff \
--aug_pred --scales  1 1.5 2 \
--flip_horizontal 

In [ ]:
########## ACFF ViT ###########

In [ ]:
# 这是我自己添加的第三个模型及其配置文件，模型的具体细节可以在 /home/aistudio/external-libraries/paddle_seg_semi1/paddleseg/models/ACFF_ViT.py 这个目录里看到
# 简要介绍一下模型、损失函数、数据增强、学习率和训练策略以及出现的一些问题
# 模型：我们采用FCN架构，以paddlecls中提供的SwinTrasnformer以及其预训练权重做为骨干网络来提取特征信息，输入数据为3×384×384的图片，四层layer我们分别提取了不同大小不同通道数的四层特征信息
     # 本次比赛的要求是生成具有目标的单通道图，一般的做法为四层特征信息的融合，即encoder-decoder架构，我们也是采用这样的方法，只不过在中间添加了特征优化的模块，可见 ACFF_ViT.py中的自优化模块（SR），注意力模块（SE）等，
     # 关于这些模块我将会在另外的方案分享里具体描述，本次提交就不做过细讲解.四层特征经过融合后使用3×3卷积调整为1通道，经过sigmoid函数与上采样恢复到原尺寸输出后在loss函数（一会细讲）和Momentum优化器的优化下进行训练.
# 损失函数：我们自己实现了iou损失函数在2分类中的实现 代码见：external-libraries/paddle_seg_semi1/paddleseg/models/losses/semi_loss.py
# 数据增强：根据paddleseg的用法，我们并没有用过多的数据增强操作，只用了resize、 对比度亮度、水平翻转、垂直反转、随即模糊、归一化这些操作，训练数据使用随机种子固定随机数，随机选取2K个作为验证集.
# 学习率：使用paddleseg推荐的poly decay 作为学习率衰减策略，初始学习率为0.01，最小学习率为0.0001，
# 训练策略以及一些问题：训练策略如下，bs = 4, 4000次迭代保存一次模型，训练2个epoch后效果最好

# 这是训练代码，复现请配置好文件后直接运行下一个代码块
%cd external-libraries/paddle_seg_semi1/
!python train.py --config configs/semi/fcn_ACFFViT.yml \
--do_eval \
--save_dir /home/aistudio/output/vit \
--save_interval 4000 \
--log_iters 100  \
--use_vdl

[Errno 2] No such file or directory: 'external-libraries/paddle_seg_semi1/'
/home/aistudio/external-libraries/paddle_seg_semi1
/home/aistudio/external-libraries/paddle_seg_semi1/paddleseg/models/ACFF_ViT.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable
2021-08-02 07:47:43 [INFO]	
------------Environment Information-------------
platform: Linux-4.4.0-150-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Cuda compilation tools, release 10.1, V10.1.243
cudnn: 7.6
GPUs used: 1
CUDA_VISIBLE_DEVICES: None
GPU: ['GPU 0: Tesla V100-SXM2-32GB']
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~16.04) 7.5.0
PaddlePaddle: 2.1.0
OpenCV: 4.1.1
------------------------------------------------


In [ ]:
# 这是预测的代码，依然是根据paddleseg套件直接运行的，采取了水平翻转的预测策略， 因为transformer固定大小的输入模式，只能固定384尺寸输入 所以没有采用多尺度预测
%cd external-libraries/paddle_seg_semi1/
!python predict.py --config configs/semi/fcn_ACFFViT.yml \
--model_path /home/aistudio/data/data99666/ACFFViT4.pdparams\
--image_path /home/aistudio/data/test_image \
--save_dir /home/aistudio/output/result_vit \
--aug_pred \
--flip_horizontal


In [ ]:
############## Next_ACFF ###############

In [ ]:
# 这是我自己添加的第四个模型及其配置文件，模型的具体细节可以在 /home/aistudio/external-libraries/paddle_seg_semi1/paddleseg/models/Next_ACFF.py 这个目录里看到
# 简要介绍一下模型、损失函数、数据增强、学习率和训练策略以及出现的一些问题
# 模型：我们采用FCN架构，以paddlecls中提供的ResNeXt101WSL以及其预训练权重做为骨干网络来提取特征信息，输入数据为3×256×256的图片，四层layer我们分别提取了256×64×64、 512×32×32 、 1024×16×16、 2048×8×8的特征信息
     # 本次比赛的要求是生成具有目标的单通道图，一般的做法为四层特征信息的融合，即encoder-decoder架构，我们也是采用这样的方法，只不过在中间添加了特征优化的模块，可见 Next_ACFF.py中的自优化模块（SR），注意力模块（SE）等，
     # 关于这些模块我将会在另外的方案分享里具体描述，本次提交就不做过细讲解.四层特征经过融合后使用3×3卷积调整为1通道，经过sigmoid函数与上采样恢复到原尺寸输出后在loss函数（一会细讲）和Momentum优化器的优化下进行训练.
# 损失函数：我们自己实现了iou损失函数在2分类中的实现 代码见：external-libraries/paddle_seg_semi1/paddleseg/models/losses/semi_loss.py
# 数据增强：根据paddleseg的用法，我们并没有用过多的数据增强操作，只用了resize、 对比度亮度、水平翻转、垂直反转、随即模糊、归一化这些操作，训练数据使用随机种子固定随机数，随机选取2K个作为验证集.
# 学习率：使用paddleseg推荐的poly decay 作为学习率衰减策略，初始学习率为0.01，最小学习率为0.0001，
# 训练策略以及一些问题：训练策略如下，2000次迭代保存一次模型，多少次迭代效果最好已经忘了，肯定不是最后一次，还是中间某次

# 这是训练代码，复现请配置好文件后直接运行下一个代码块
%cd external-libraries/paddle_seg_semi1/
!python train.py --config configs/semi/fcn_NextACFF.yml \
--do_eval \
--save_dir /home/aistudio/output/next \
--save_interval 2000 \
--log_iters 100  \
--use_vdl

In [ ]:
# 这是预测的代码，依然是根据paddleseg套件直接运行的，采取了多尺度预测和水平翻转的预测策略， 这里就没什么好讲的了
%cd external-libraries/paddle_seg_semi1/
!python predict.py --config configs/semi/fcn_NextACFF.yml \
--model_path /home/aistudio/data/data99666/NextACFF.pdparams \
--image_path /home/aistudio/data/test_image/ \
--save_dir /home/aistudio/output/result_acffnext \
--aug_pred --scales  1 1.5 2 \
--flip_horizontal 

In [1]:
# 另外说一下，我们根据比赛的评价指标，修改了 external-libraries/paddle_seg_semi1/paddleseg/core/train.py、 infer.py、 predict.py、 val.py等文件
# 但不影响运行，主要修改为：修改了输入输出的数据类型 int -> float32   修改了输出 多保存了图像的npy文件用于后续的模型融合

In [ ]:
# 这个融合策略是上述四个模型结果取平均融合的方法，B榜结果8.0383 不是最好的结果  可以忽略， 只是我不想删，因为是我队友辛苦写出来的

import os
import numpy as np
import cv2
import glob

from paddleseg.utils import logger, progbar

save_image_path = '/home/aistudio/output/result_ensem/pre'
if not os.path.exists(save_image_path):
    os.makedirs(save_image_path)

logits_path = [
    '/home/aistudio/output/result_acff/logits',
    '/home/aistudio/output/result_fmf/logits',
    '/home/aistudio/output/result_vit/logits',
    '/home/aistudio/output/result_acffnext/logits'
]
npy_list = glob.glob(os.path.join(logits_path[0], '*.npy'))
npy_list = [npy.split('/')[-1] for npy in npy_list]

progbar_pred = progbar.Progbar(target=len(npy_list), verbose=1)

weights = [0.2, 0.2, 0.2, 0.4]

for j, npy in enumerate(npy_list):
    final_logit = 0
    for i, path in enumerate(logits_path):
        if 'vit' in path:
            logit = np.load(os.path.join(path, npy)) / 2.0
        else:
            logit = np.load(os.path.join(path, npy)) / 6.0
        final_logit += logit

    final_logit = final_logit / len(logits_path)
    pre = final_logit#[:, 1, :, :]
    pre = pre[0,:,:]
    pre = np.transpose(pre, [1, 2, 0]).squeeze(axis=-1) * 255
    
    cv2.imwrite(os.path.join(save_image_path, npy.split('.')[0] + '.jpg'), pre)
    progbar_pred.update(j)

/home/aistudio/external-libraries/paddle_seg_semi1/paddleseg/models/ACFF_ViT.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable
10985/10989 [============================>.] - ETA: 0

In [ ]:
import os
len(os.listdir('/home/aistudio/output/result_ensem/pre'))

10989

In [ ]:
# 这个融合策略是上述四个模型结果加权融合的方法，B榜最好结果8.0428 达到最好的结果 
import os
import numpy as np
import cv2
import glob

from paddleseg.utils import logger, progbar

save_image_path = '/home/aistudio/output/result_ensem_weight/pre'
if not os.path.exists(save_image_path):
    os.makedirs(save_image_path)

logits_path = [
    '/home/aistudio/output/result_acff/logits',
    '/home/aistudio/output/result_fmf/logits',
    '/home/aistudio/output/result_vit/logits',
    '/home/aistudio/output/result_acffnext/logits'
] # 这是上述四个模型产生的图像npy文件


npy_list = glob.glob(os.path.join(logits_path[0], '*.npy'))
npy_list = [npy.split('/')[-1] for npy in npy_list]

progbar_pred = progbar.Progbar(target=len(npy_list), verbose=1) # paddleseg自带的进度条

weights = [0.4, 0.2, 0.2, 0.2] # 四个npy文件的权重

for j, npy in enumerate(npy_list):
    final_logit = 0
    for i, path in enumerate(logits_path):
        if 'vit' in path:
            logit = np.load(os.path.join(path, npy)) / 2.0  # 这是transformer为骨干网络预测的结果 /2.0是因为加了水平翻转预测
        else:
            logit = np.load(os.path.join(path, npy)) / 6.0 # 这是其他三个模型预测的结果 /6.0是因为加了水平翻转预测 以及三个尺度预测 即2*3 = 6
        final_logit = final_logit +  logit * weights[i] # 加权融合

  #  final_logit = final_logit / len(logits_path)
    pre = final_logit#[:, 1, :, :]
    pre = pre[0,:,:]
    pre = np.transpose(pre, [1, 2, 0]).squeeze(axis=-1) * 255
    
    cv2.imwrite(os.path.join(save_image_path, npy.split('.')[0] + '.jpg'), pre) # 输出
    progbar_pred.update(j)

In [ ]:
# 第一次  [0.4, 0.2, 0.2, 0.2] finished 结果记录 可忽略
# 第二次 [0.2, 0.4, 0.2, 0.2] finished
# 第三次 [0.2, 0.2, 0.4, 0.2] finished
# 第四次 [0.2, 0.2, 0.2, 0.4] finished
# 第五次 [0.5, 0.2, 0.15, 0.15] finished
# 第六次 [0.6, 0.2, 0.1, 0.1] finished
# 第七次 [0.5, 0.3, 0.1, 0.1] finished
# 第八次 weights = [0.5, 0.1, 0.2, 0.2]